<a href="https://colab.research.google.com/github/vs-152/FL-Contributions-Incentives-Project/blob/main/ISO_CIFAR10_OR_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pulp
import copy
import time
from sklearn.model_selection import StratifiedShuffleSplit
import torchvision
# from torchvision.datasets import CIFAR10
# import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
from itertools import chain, combinations
from tqdm import tqdm
from scipy.special import comb
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import glob, nibabel as nib, pandas as pd
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, ScaleIntensityd,
    RandFlipd, RandSpatialCropd, Compose, SelectItemsd
)


In [2]:
from utils import *
from models import *


In [3]:
# -----------------------------------------------------------
# 0.  Paths & meta-data
# -----------------------------------------------------------

# BRATS_DIR   = "/users/adcw447/archive/FeTS2022/MICCAI_FeTS2022_TrainingData"
# VAL_DIR     = "/users/adcw447/archive/FeTS2022/MICCAI_FeTS2022_ValidationData"
BRATS_DIR   = "/mnt/c/Datasets/MICCAI_FeTS2022_TrainingData"
VAL_DIR     = "/mnt/c/Datasets/MICCAI_FeTS2022_ValidationData"
CSV_PATH    = f"{BRATS_DIR}/partitioning_1.csv"     # pick 1, 2 … or sanity
MODALITIES  = ["flair", "t1", "t1ce", "t2"]
LABEL_KEY   = "seg"  # BraTS tumour mask filename ending

# -----------------------------------------------------------
# 1.  Read partition file → mapping   {client_id: [subjIDs]}
# -----------------------------------------------------------
part_df          = pd.read_csv(CSV_PATH)
partition_map    = (
    part_df.groupby("Partition_ID")["Subject_ID"]
           .apply(list)
           .to_dict()
)
NUM_CLIENTS = len(partition_map)

# -----------------------------------------------------------
# 2.  Build a list of dicts – one per subject
# -----------------------------------------------------------
def build_records(subject_ids):
    recs = []
    for sid in subject_ids:
        subj_dir = f"{BRATS_DIR}/{sid}"
        rec = {m: f"{subj_dir}/{sid}_{m}.nii.gz"
               for m in MODALITIES}
        rec["seg"] = f"{subj_dir}/{sid}_{LABEL_KEY}.nii.gz"
        
        recs.append(rec)
    return recs

def build_val_records(val_dir):
    subjects = sorted(glob.glob(f"{val_dir}/FeTS2022_*_flair.nii.gz"))
    recs = []
    for flair_path in subjects:
        sid = flair_path.split("/")[-1].split("_flair")[0]
        subj_dir = f"{val_dir}/{sid}"
        rec = {m: f"{subj_dir}/{sid}_{m}.nii.gz" for m in MODALITIES}
        rec["seg"] = f"{subj_dir}/{sid}_{LABEL_KEY}.nii.gz"   #  ← NEW
        recs.append(rec)
    return recs

# -----------------------------------------------------------
# 3.  MONAI transform pipelines  (fixed)
# -----------------------------------------------------------
IMG_KEYS   = [m for m in MODALITIES]
ALL_KEYS   = IMG_KEYS + [LABEL_KEY]

train_tf = Compose([
    LoadImaged(keys=ALL_KEYS),
    EnsureChannelFirstd(keys=ALL_KEYS),
    Orientationd(keys=ALL_KEYS, axcodes="RAS"),
    ScaleIntensityd(keys=ALL_KEYS, minv=-1.0, maxv=1.0), # scale to [-1,1]. Diffusion Models do better if centered on a 0 mean
    SelectItemsd(keys=ALL_KEYS),
])

val_tf = Compose([
    LoadImaged(keys=ALL_KEYS),
    EnsureChannelFirstd(keys=ALL_KEYS),
    Orientationd(keys=ALL_KEYS, axcodes="RAS"),
    ScaleIntensityd(keys=MODALITIES, minv=-1.0, maxv=1.0),  #  ← OK: masks stay untouched
    SelectItemsd(keys=ALL_KEYS),
])


In [4]:

# -----------------------------------------------------------
# 4.  Build per-client datasets & dataloaders
# -----------------------------------------------------------
# CUT_OFF    = 4  # How many sites to discard.
# train_datasets = {}     # {client_id: monai CacheDataset}
# for cid, subj_list in partition_map.items():
#     if cid > CUT_OFF:
#         print(f"capping at {cid} for now")
#         break
#     records = build_records(subj_list)
#     train_datasets[cid] = CacheDataset(data=records, transform=train_tf, cache_rate=0.2)

# --- SUBSAMPLE FOR DEV TESTING

import random                    # NEW ─ reproducible subsampling
FRAC = 0.3 #.10                      # 10 % of every client’s cases
SEED = 42                        # set to None for pure randomness
rng = random.Random(SEED)        # independent RNG so you don’t disturb numpy

# -----------------------------------------------------------
# 4.  Build per-client datasets & dataloaders
# -----------------------------------------------------------
CUT_OFF = 4
train_datasets = {}

for cid, subj_list in partition_map.items():
    if cid > CUT_OFF:
        print(f"capping at {cid} for now")
        break

    # ── pick 10 % of this client’s subjects ─────────────────
    k = max(1, int(len(subj_list) * FRAC))        # always keep ≥1 case
    sample_ids = rng.sample(subj_list, k)

    # build dataset from that subset
    records = build_records(sample_ids)
    train_datasets[cid] = CacheDataset(data=records, transform=train_tf, cache_rate=1)

print({cid: len(ds) for cid, ds in train_datasets.items()})
# e.g. {1: 25, 2: 1, 3: 2, 4: 5}


Loading dataset: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:11<00:00,  1.25it/s]

capping at 5 for now
{1: 153, 2: 1, 3: 4, 4: 14}


In [5]:
# -----------------------------------------------------------
# 7.  Build test dataset & dataloader
# -----------------------------------------------------------
val_records  = build_val_records(VAL_DIR)
test_dataset  = CacheDataset(data=val_records, transform=val_tf, cache_rate=1)
# test_loader   = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)

In [6]:
# def test_inference(model, test_dataset):
#     # --------- INFERENCE FOR SEGMENTATION
#     model.eval()
#     total_dice = 0.0
#     testloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

#     with torch.no_grad():
#         for batch in testloader:
#             images = torch.stack([batch[k] for k in ["flair", "t1", "t1ce", "t2"]], dim=1).squeeze(2).to(device)  # (B, 4, D, H, W)
#             labels = batch["seg"].long().to(device)  # (B, 1, D, H, W)

#             outputs = model(images)
#             preds = torch.argmax(outputs, dim=1, keepdim=True)  # (B, 1, D, H, W)

#             # Simple Dice calculation (per class optional)
#             intersection = (preds == labels).sum().item()
#             total = labels.numel()
#             total_dice += 2.0 * intersection / (total + preds.numel())

#     return total_dice / (len(testloader) + 0.0000001)

def test_inference(model, test_dataset):
    """
    Computes mean Dice across the three BraTS channels
    (whole tumour, tumour-core, enhancing-core).

    Returns
    -------
    float   ─ mean Dice in the range [0, 1]
    """
    model.eval()
    loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    dice_sum = torch.zeros(3, device=device)   # accumulate per-class Dice
    n_batches = 0

    with torch.no_grad():
        for batch in loader:
            # ---- image tensor ------------------------------------------------
            img = torch.cat([batch[k] for k in ("flair", "t1", "t1ce", "t2")],
                            dim=1).to(device)                     # (B,4,D,H,W)

            # ---- ground-truth mask → one-hot (B,3,D,H,W) ---------------------
            raw    = batch["seg"].squeeze(1).cpu().numpy()       # (B,D,H,W)
            target = torch.tensor(
                        preprocess_mask_labels(raw),
                        dtype=torch.float32, device=device
                     )

            # ---- network prediction → one-hot (B,3,D,H,W) --------------------
            logits  = model(img)                                 # (B,3,D,H,W)
            pred_ch = torch.argmax(logits, dim=1)                # (B,D,H,W)
            pred    = torch.nn.functional.one_hot(
                          pred_ch, num_classes=3
                      ).permute(0,4,1,2,3).float()

            # ---- per-class Dice ---------------------------------------------
            intersect = 2 * (pred * target).sum(dim=(2,3,4))     # (B,3)
            denom     = (pred + target).sum(dim=(2,3,4)) + 1e-6
            dice_sum += (intersect / denom).squeeze(0)           # add (3,)
            n_batches += 1

    return (dice_sum / n_batches).mean().item()                  # scalar Dice



In [18]:
# N = 10 #srch

# after you build train_datasets
idxs_users = np.array(sorted(train_datasets.keys()))
N = len(idxs_users)

# N = list(train_datasets.keys())[-1]
print(f"We got {N} clients")
local_bs = 2
lr = 1e-4
local_ep = 1
EPOCHS = 1

# noise_rates = np.linspace(0, 1, N, endpoint=False)
# split_dset = mnist_iid(trainset, N)
# user_groups = {i: 0 for i in range(1, N+1)}
# noise_idx = {i: 0 for i in range(1, N+1)}
# train_datasets = {i: 0 for i in range(1, N+1)}
# for n in range(N):
#     user_groups[n+1] = np.array(list(split_dset[n]), dtype=np.int)
#     user_train_x, user_train_y = x_train[user_groups[n+1]], y_train[user_groups[n+1]]
#     user_noisy_y, noise_idx[n+1] = noisify_MNIST(noise_rates[n], 'symmetric', user_train_x, user_train_y)
    
#     train_datasets[n+1] = CustomTensorDataset((user_train_x, user_noisy_y), transform_train)

def copy_batchnorm_stats(subset_weights, global_model_state_dict):
    for pair_1, pair_2 in zip(subset_weights.items(), global_model_state_dict.items()):
        if ('running' in pair_1[0]) or ('batches' in pair_1[0]):
            subset_weights[pair_1[0]] = global_model_state_dict[pair_1[0]]
    
    return subset_weights


global_model = ResUNet3D(in_channels=4, out_channels=3).to(device) # Segmentation model used in FeTS

global_model.to(device)
global_model.train()

global_weights = global_model.state_dict()
powerset = list(powersettool(range(1, N+1)))[1:]          # discard ()
# … the rest unchanged …


# submodel_dict = {}
# submodel_dict[()] = copy.deepcopy(global_model)
# Change instead to storing the submodel weights to disk using torch.save
import os
submodel_dir = "submodels"
os.makedirs(submodel_dir, exist_ok=True)
submodel_file_template = os.path.join(submodel_dir, "submodel_{}.pth")
global_model_path = os.path.join(submodel_dir, "global_model.pth")
# Save the global model weights
torch.save(global_weights, global_model_path)

accuracy_dict = {}
# accuracy_dict[()] = 0.0                                # utility of empty coalition
shapley_dict = {}

We got 4 clients


In [19]:
powerset

[(1,),
 (2,),
 (3,),
 (4,),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 (1, 2, 3, 4)]

In [21]:
start_time = time.time()

# Store initial single user submodels as the initial global model
for user in range(1, N+1):
    model_path = submodel_file_template.format(user)
    # global_weights is already global_model.state_dict()  
    torch.save(global_weights, model_path)
 
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
print_every = 1

idxs_users = np.arange(1, N+1)
# total_data = sum(len(user_groups[i]) for i in range(1, N+1))
# fraction = [len(user_groups[i])/total_data for i in range(1, N+1)]

# ── collect dataset sizes ──────────────────────────────────────────────────
# MONAI's CacheDataset inherits __len__, so `len(ds)` is cheap:
sizes = {k: len(ds) for k, ds in train_datasets.items()}


# ── total samples across all clients ───────────────────────────────────────
total_data = sum(sizes.values())

# ── FedAvg weight (a.k.a. fraction) for each client ────────────────────────
# Keep the list in key order 1…N so it lines up with your loops later.
fraction = [sizes[i] / total_data for i in range(1, N + 1)]

# ───────────────────────────────────────────────────────────────────────────

for epoch in tqdm(range(EPOCHS)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')
    global_model.train()
    for idx in tqdm(idxs_users):
        trainloader = DataLoader(train_datasets[idx], batch_size=local_bs, shuffle=True,
                                 num_workers=4, pin_memory=True, persistent_workers=True)
        local_trainer = LocalUpdateMONAI(
            lr=lr,
            local_ep=local_ep,
            trainloader=trainloader,
            img_keys=("flair","t1","t1ce","t2"),
            label_key="seg",
        )
        
        local_model = copy.deepcopy(global_model)  # Ensure local_model is a fresh copy
        w, loss = local_trainer.update_weights(model=local_model)
        local_weights.append(w)
        local_losses.append(copy.deepcopy(loss))
    del local_model  # Free memory after each client update
    global_weights = average_weights(local_weights, fraction) # This operates on a list of state_dicts
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # calculate weight updates aka pseudo-gradients Delta_i = local_weight - old_global_weight
    gradients = calculate_gradients(local_weights, global_model.state_dict())

    # update the single user submodels stored on disk based on the pseudo-gradients
    for i in range(1, N+1):
        user_path = submodel_file_template.format(i)
        prev_user_weights = torch.load(user_path)  # Load previous user weights
        user_weights = update_weights_from_gradients(gradients[i-1], prev_user_weights)
        
        # unsure the following line is needed in this context
        # subset_weights = copy_batchnorm_stats(subset_weights, global_model.state_dict())
        
        # Save the updated user weights back to disk
        torch.save(user_weights, user_path)

    # update the global model with the averaged weights
    global_model.load_state_dict(global_weights)
    global_model.eval()

    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        # print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

test_dsc = test_inference(global_model, test_dataset)
print(f' \n Results after {EPOCHS} global rounds of training:')
print("|---- Test DSC: {:.2f}%".format(100*test_dsc))

accuracy_dict[powerset[-1]] = test_dsc


  0%|                                                                                                                                            | 0/1 [00:00<?, ?it/s]


 | Global Training Round : 1 |




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:16<00:00, 136.89s/it]

 
Avg Training Stats after 1 global rounds:
Training Loss : 5.658808510121587
 
 Results after 1 global rounds of training:
|---- Test DSC: nan%


In [14]:
len(subset)

0

In [15]:

# ADJUSTED-OR APPROX
for subset in powerset[:-1]: 
    if len(subset) > 1:
        # calculate the average of the subset of weights from list of all the weights in the subset
        subset_weights = average_weights([torch.load(submodel_file_template.format(user)) for user in subset], [fraction[i-1] for i in subset])
    elif len(subset) == 1:
        # for single user submodels, just load the weights from disk, no need to average
        subset_weights = torch.load(submodel_file_template.format(subset[0]))
        # need to make a model from the averaged weights to test it
    else:
        continue
    submodel = ResUNet3D(in_channels=4, out_channels=3).to(device)
    submodel.load_state_dict(subset_weights)   
    test_dsc = test_inference(submodel,test_dataset)
    print(f' \n Results after {EPOCHS} global rounds of training (for OR): ')
    print("|---- Test DSC for {}: {:.2f}%".format(subset, 100*test_dsc))
    accuracy_dict[subset] = test_dsc
    del submodel  # Free memory after each subset test

trainTime = time.time() - start_time
start_time = time.time()
shapley_dict = shapley(accuracy_dict, N)
shapTime = time.time() - start_time
start_time = time.time()
lc_dict = least_core(accuracy_dict, N)
LCTime = time.time() - start_time
totalShapTime = trainTime + shapTime
totalLCTime = trainTime + LCTime
print(f'\n ACCURACY: {accuracy_dict[powerset[-1]]}')
print('\n Total Time Shapley: {0:0.4f}'.format(totalShapTime))
print('\n Total Time LC: {0:0.4f}'.format(totalLCTime))

 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (1,): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (2,): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (3,): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (4,): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (1, 2): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (1, 3): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (1, 4): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (2, 3): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (2, 4): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (3, 4): nan%
 
 Results after 1 global rounds of training (for OR): 
|---- Test DSC for (1, 2, 3): nan%
 
 Results after 1 global rounds of t

PulpSolverError: Pulp: Error while executing /home/aklina/miniconda3/envs/m_quant/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc

In [ ]:
print("Shapley allocation:")
for cid, phi in shapley_dict.items():
    print(f" client {cid}: {phi:.4f}")

print("\nLeast-Core allocation:")
for var in lc_dict.variables():
    if var.name.startswith("x("):
        print(f" client {var.name[2:-1]}: {var.value():.4f}")
print(f" e (slack): {lc_dict.variablesDict()['e'].value():.4f}")


In [ ]:
def stats(vector):
    n = len(vector)
    egal = np.array([1/n for i in range(n)])
    normalised = np.array(vector / vector.sum())
    msg = f'Original vector: {vector}\n'
    msg += f'Normalised vector: {normalised}\n'
    msg += f'Max Dif: {normalised.max()-normalised.min()}\n'
    msg += f'Distance: {np.linalg.norm(normalised-egal)}\n'

    msg += f'Budget: {vector.sum()}\n'
    print(msg)

In [ ]:
stats(np.array(list(shapley_dict.values())))

In [ ]:
stats(np.array([i.value() for i in lc_dict.variables()])[1:])